In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
from collections import defaultdict
from multiprocessing import Manager
import sklearn

In [2]:
spark = (
    SparkSession.builder.appName('Insights and Findingss')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '7g')
    .config('spark.executor.memory', '3g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/08 17:13:29 WARN Utils: Your hostname, DESKTOP-F216TKE resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/08 17:13:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 17:13:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/08 17:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/08 17:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/08 17:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53520)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(

In [3]:
sdf = spark.read.parquet(".././data/curated/knntransactions")

In [4]:
sdf.show()

24/10/08 17:13:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+----+--------+--------------+------------+-------+-----------+------------------+--------------------+-------------+---------------+-----+------+------------------+--------------------+--------------------+----+---------+--------------------------+----------+-------------+--------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+-----+-----+-----+------------------+-----------+------------+--------------+--------------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|consumer_name|        address|state|gender| fraud_probability|       merchant_name|                tags|type|take_rate|merchant_fraud_probability|AREASQKM21|   SHAPE_Leng|    SHAPE_Area|               lon|                lat|Median_age_persons|Median_tot_fam_inc_weekly|Median_tot_hhd_inc_weekly|Average_household_size|Tot_M|Tot_P|Tot_F|  expected_revenue|day_of_week|da

In [5]:
sdf = sdf.drop('expected_revenue')

In [6]:
sdf = sdf.withColumn('expected_revenue', 
                                       sdf.dollar_value
                                       * (1.0 - sdf.merchant_fraud_probability / 100.0) 
                                       * (1.0 - sdf.fraud_probability / 100.0) 
                                       * sdf.take_rate / 100.0)

Calculate the Range of Fraud Probabilites along with outliers
merchant_stats.pandas_api().plot.scatter

In [7]:
sdf.printSchema()

root
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- consumer_id: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- consumer_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- fraud_probability: double (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- type: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- merchant_fraud_probability: double (nullable = true)
 |-- AREASQKM21: double (nullable = true)
 |-- SHAPE_Leng: double (nullable = true)
 |-- SHAPE_Area: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- 

In [7]:
average_fraud = sdf.groupBy('merchant_abn').agg(F.avg('merchant_fraud_probability').alias('average_fraud'))

In [9]:
average_fraud

merchant_abn,average_fraud
60978195146,31.295725237683875
67165527084,28.716135880472198
21552853184,29.146657690001934
33651513345,49.18069316283151
97088280855,32.69119010061531
36866208934,29.140608059909226
31245723081,34.48038113786465
53074973777,34.014680525406185
11633090957,28.41785081839181
95824231566,35.945004525346235


In [9]:
pdf = average_fraud.pandas_api()
pdf['average_fraud'].plot.density(bw_method = 0.2, color_discrete_sequence = ['green'] )


In [11]:
pdf['average_fraud'].plot.box()

Remove Merchants with a High Fraud Probability

In [12]:
high_merchant_fraud = average_fraud.filter(F.col('average_fraud') > 51.12865)

In [13]:
high_merchant_fraud.select('merchant_abn').distinct().count()

266

Remove the merchants with a high fraud rate 

In [14]:
merchants_high = high_merchant_fraud.select('merchant_abn')

In [15]:
merchants_high = merchants_high.toPandas()
merchants_high = merchants_high['merchant_abn'].tolist()

In [16]:
sdf = sdf.filter(~sdf['merchant_abn'].isin(merchants_high))

In [17]:
sdf.count()

10289310

In [18]:
len(merchants_high)

266

There are 4026-266 merchants left

Find average consumer fraud for each merchant and remove those with high fraud values 

In [19]:
df = spark.read.parquet('.././data/curated/filtered')

In [20]:
df.show()

+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
|merchant_abn|                tags|take_rate|type|     Total Revenue|Total Sales|    Merchant Fraud|    Consumer Fraud|Median Family Income|avg(seasonal_sales)|avg(original_sales)|Average Household Size|Unique Customers|Average Expected Revenue|Days with Sales|
+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
| 28687004922|computer programm...|     3.88|   b|1199539.6177607474|      11884|29.145120033052184|14.219447100047875|  1976.8873274991586|  6432.504930999647|  6422.561098956571|    2.4543840457758335|           

In [21]:
df.count()

4026

In [22]:
pdf = df.pandas_api()
pdf['Consumer Fraud'].plot.density(bw_method = 0.2)

In [23]:
pdf['Consumer Fraud'].plot.box()

In [24]:
high_consumer = df.filter(F.col('Consumer Fraud') > 17.46632)

In [25]:
high_consumer = high_consumer.toPandas()['merchant_abn'].tolist()

In [26]:
len(high_consumer)

204

In [27]:
sdf = sdf.filter(~sdf['merchant_abn'].isin(high_consumer))

In [28]:
tags = sdf.groupBy('tags').agg(F.sum('dollar_value').alias('Total Revenue'), F.count('*').alias('Total Sales')).sort('Total Sales')

In [29]:
tags.count()

25

In [30]:
tags.pandas_api().plot.bar('tags', 'Total Sales')

In [31]:
tags.pandas_api().plot.bar('tags', 'Total Revenue')

In [32]:
tags_types = tags.select(F.col('tags'))

In [33]:
tags_types = tags_types.toPandas()['tags'].tolist()

In [34]:
tags_types

['jewelry, watch, clock, silverware',
 'art dealers, galleries',
 'telecom',
 'equipment, tool, furniture, appliance rental, leasing',
 'antique, repairs, restoration',
 'stationery, office supplies, printing, writing paper',
 'florists supplies, nursery stock, flowers',
 'bicycle,',
 'health, beauty spas',
 'hobby, toy, game',
 'motor vehicle supplies, new parts',
 'music, musical instruments, pianos, sheet music',
 'books, periodicals, newspapers',
 'lawn, garden supply outlets, including nurseries',
 'shoe',
 'artist supply, craft',
 'furniture, home furnishings, equipment, manufacturers',
 'computer programming, data processing, integrated systems design',
 'computers, computer peripheral equipment, software',
 'opticians, optical goods, eyeglasses',
 'cable, satellite, other pay television, radio',
 'watch, clock, jewelry repair',
 'digital goods, books, movies, music',
 'gift, card, novelty, souvenir',
 'tent, awning']

Feature Engineering Transaction Parquet

In [35]:
sdf2 = sdf.groupBy('merchant_abn', 'tags', 'take_rate', 'type').agg(F.sum('dollar_value').alias('Total Revenue'), F.count('*').alias('Total Sales'), F.avg('merchant_fraud_probability').alias('Merchant Fraud'), F.avg('fraud_probability').alias('Consumer Fraud'), F.avg('Median_tot_fam_inc_weekly').alias('Median Family Income'), F.avg('seasonal_sales'), F.avg('original_sales'), F.avg('Average_household_size').alias('Average Household Size'), F.countDistinct('consumer_id').alias('Unique Customers'), F.avg('expected_revenue').alias('Average Expected Revenue'), F.countDistinct('order_datetime').alias('Days with Sales'))

In [36]:
sdf2.show()

+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
|merchant_abn|                tags|take_rate|type|     Total Revenue|Total Sales|    Merchant Fraud|    Consumer Fraud|Median Family Income|avg(seasonal_sales)|avg(original_sales)|Average Household Size|Unique Customers|Average Expected Revenue|Days with Sales|
+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
| 36413911362|opticians, optica...|     4.84|   b| 93552.15471576259|       1202|28.585647608037544|15.327178089127178|  1975.4567387687189|  6544.434026622296|  6553.729450915142|     2.446089850249584|           

In [37]:
sdf2.write.mode('overwrite').parquet('.././data/curated/filtered')

In [38]:
sdf.write.mode('overwrite').parquet('.././data/curated/knntransactions2')

In [39]:
types = sdf.groupBy('type', 'tags', 'merchant_abn').agg(F.count('*'))

In [40]:
types = types.toPandas()
types.to_csv('types.csv')

In [41]:
#sdf2 = sdf.groupBy('order_datetime','merchant_abn', 'tags', 'take_rate', 'type', 'expected_revenue').agg(F.sum('dollar_value').alias('Total Revenue'), F.count('*').alias('Total Sales'), F.avg('merchant_fraud_probability').alias('Merchant Fraud'), F.avg('fraud_probability').alias('Consumer Fraud'), F.avg('Median_tot_fam_inc_weekly').alias('Median Family Income'), F.avg('seasonal_sales'), F.avg('original_sales'), F.avg('Average_household_size').alias('Average Household Size'), F.countDistinct('consumer_id').alias('Unique Customers'))

24/10/05 19:16:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1207679 ms exceeds timeout 120000 ms
24/10/05 19:16:01 WARN SparkContext: Killing executors is not supported by current scheduler.
24/10/05 19:16:02 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 